<a href="https://colab.research.google.com/github/Jonathangadeaharder/GameTheoryMA/blob/main/True_Experimente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import random
import copy
def mutate(s, m):
    """Flips each bit of s with a probability of 1/m."""
    return [bit if random.random() > 1/m else 1-bit for bit in s]

def calculate_hamming_distance(sol1, sol2):
    """Calculate the Hamming distance between two solutions."""
    ham = sum(el1 != el2 for el1, el2 in zip(sol1, sol2))
    return ham

def calculate_diversity(P):
    """
    Calculates the diversity of the population, considering only unique solutions.

    Parameters:
    P (list): The population, each solution is a list of indices.

    Returns:
    int: The total pairwise Hamming distance among all unique solutions.
    """
    # Remove duplicates and consider only unique solutions
    unique_solutions = set(tuple(sol) for sol in P)
    unique_solutions_list = list(unique_solutions)  # Convert back to list for indexing

    diversity = 0
    for i in range(len(unique_solutions_list)):
        for j in range(i + 1, len(unique_solutions_list)):
            diversity += calculate_hamming_distance(unique_solutions_list[i], unique_solutions_list[j])

    return diversity

def is_valid_maximum_matching(s, l, r):
    """
    Check if the solution 's' represents a valid maximum matching in a bipartite graph.

    Parameters:
    s (list): Solution represented as a binary string of length r*l.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.

    Returns:
    bool: True if the solution represents a valid maximum matching, False otherwise.
    """
    matched_right = set()
    for i in range(r):
        matched_left = -1
        for j in range(l):
            if s[i * l + j] == 1:
                if matched_left != -1:  # More than one match for a right vertex
                    return False
                matched_left = j
        if matched_left != -1:
            if matched_left in matched_right:  # Duplicate match found
                return False
            matched_right.add(matched_left)

    # Check if all right vertices are matched (for maximum matching)
    return len(matched_right) == r


def generate_initial_population(mu, l, r):
    """
    Generates an initial population of solutions where each r_i is mapped to l_i for i=0 to i=r.
    """
    m = l * r
    initial_population = []
    for _ in range(mu):
        solution = [0] * m
        for i in range(min(l, r)):  # To handle cases where l != r
            index = i * l + i  # Mapping r_i to l_i
            solution[index] = 1
        initial_population.append(solution)
    return initial_population


def print_solution_as_indices(s, l, r):
    """
    Prints a single solution as an array of indices showing the matchings.

    Parameters:
    s (list): The solution to print, represented as a binary string of length r*l.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.
    """
    for i in range(r):
        matched_index = -1  # Default value if no match is found
        for j in range(l):
            if s[i * l + j] == 1:
                matched_index = j
                break
        print(matched_index, end=" ")
    print()  # New line at the end of the solution

def print_population_as_matrix(P, l, r):
    """
    Prints the entire population as a matrix, each row representing a solution.

    Parameters:
    P (list): The population to print, each solution is a binary string of length r*l.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.
    """
    for solution in P:
        print_solution_as_indices(solution, l, r)

def ead_algorithm_for_matching(mu, l, r, log=False):
    """
    Implementation of the EAD algorithm adapted for a bipartite graph matching problem.

    Parameters:
    mu (int): Population size.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.
    log (bool): If set to True, logs the progress of the algorithm.

    Returns:
    tuple: A tuple containing the final population and the number of fitness evaluations.
    """
    P = generate_initial_population(mu, l, r)
    fitness_evaluations = 0
    m = l * r
    while calculate_diversity(P) < mu * r * (mu-1):
        s = random.choice(P)
        s_prime = mutate(s, m)
        if is_valid_maximum_matching(s_prime, l, r) and s != s_prime:
            P.append(s_prime)
            fitness_evaluations += 1
            # Calculate and print contributions for each solution including s_prime
            contributions = []
            total_div = calculate_diversity(P)
            for i in range(len(P)):
                tmp = P[i]
                P.pop(i)
                div = calculate_diversity(P)
                P.insert(i,tmp)
                contr = total_div-div
                contributions.append((P[i],contr))
            if log:
                print("\nContributions:")
                for sol, contrib in contributions:
                    print_solution_as_indices(sol, l, r)
                    print(f": {contrib}")
                print("\n")

            # Find and remove the solution with the minimum contribution
            min_contribution = min(contributions, key=lambda x: x[1])[1]
            min_contributors = [sol for sol, contrib in contributions if contrib == min_contribution]
            z = random.choice(min_contributors)
            P.remove(z)
            # Print current diversity and fitness evaluations
            if log:
                print(f"Current Diversity: {calculate_diversity(P)}, Fitness Evaluations: {fitness_evaluations}")

    return P, fitness_evaluations
for i in range(3):
    # Example usage
    l, r = 8, 7  # Number of vertices on left and right
    mu = 3  # Population size
    print(l,r,mu)
    population, fitness_evaluations = ead_algorithm_for_matching(mu, l, r,log=False)
    print(f"Fitness Evaluations: {fitness_evaluations}")
    print_population_as_matrix(population,l,r)
    print(calculate_diversity(population))
for i in range(3):
    # Example usage
    l, r = 11, 7  # Number of vertices on left and right
    mu = 3  # Population size
    print(l,r,mu)
    population, fitness_evaluations = ead_algorithm_for_matching(mu, l, r,log=False)
    print(f"Fitness Evaluations: {fitness_evaluations}")
    print_population_as_matrix(population,l,r)
    print(calculate_diversity(population))

8 7 3
Fitness Evaluations: 48
6 1 2 4 5 7 3 
2 0 6 1 7 4 5 
5 3 7 6 4 0 1 
42
8 7 3
Fitness Evaluations: 44
6 1 2 3 4 7 5 
5 3 4 6 7 0 2 
2 5 7 4 0 1 6 
42
8 7 3
Fitness Evaluations: 57
0 4 7 3 5 6 2 
7 6 3 2 0 4 5 
1 3 5 7 4 2 6 
42
11 7 3
Fitness Evaluations: 34
2 9 3 1 0 5 6 
1 8 4 3 9 0 10 
8 10 2 5 4 7 3 
42
11 7 3
Fitness Evaluations: 36
5 2 9 7 8 0 10 
0 10 8 6 4 5 7 
3 1 7 10 2 9 0 
42
11 7 3
Fitness Evaluations: 33
0 8 5 4 6 7 9 
9 1 2 3 7 4 0 
10 7 9 6 0 1 2 
42
